In [1]:
import importlib
import os
from pathlib import Path
import sys

from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS
from dotenv import load_dotenv, find_dotenv
import pandas as pd

from dm import Country, DemographicModeling
from dm.utils import get_countries, add_enrich_aliases

# import arcpy if available
if importlib.util.find_spec("arcpy") is not None:
    import arcpy

In [2]:
# paths to common data locations - NOTE: to convert any path to a raw string, simply use str(path_instance)
dir_prj = Path.cwd().parent

dir_data = dir_prj/'data'
dir_raw = dir_data/'raw'
dir_int = dir_data/'interim'

gdb_raw = dir_raw/'raw.gdb'
gdb_int = dir_int/'interim.gdb'

# import the project package from the project package path - only necessary if you are not using a unique environemnt for this project
sys.path.append(str(dir_prj/'src'))
import canada_modeling

# load the "autoreload" extension so that code can change, & always reload modules so that as you change code in src, it gets loaded
%load_ext autoreload
%autoreload 2

# load environment variables from .env
load_dotenv(find_dotenv())

True

In [3]:
get_countries('local')

,geo_ref,country,year
0,CAN_ESRI_2019,CAN,2019
1,USA_ESRI_2019,USA,2019
2,USA_ESRI_2020,USA,2020


In [4]:
can = Country('CAN')

can

<dm.Country - CAN (local)>

In [5]:
evars = can.enrich_variables

evars

,name,alias,type,vintage,data_collection,enrich_str,enrich_field_name
0,A16AITFNAT,2016 First Nations Single Ident,COUNT,2016,AboriginalIdentity,AboriginalIdentity.A16AITFNAT,AboriginalIdentity_A16AITFNAT
1,A16AITIDT,2016 Aboriginal Identity,COUNT,2016,AboriginalIdentity,AboriginalIdentity.A16AITIDT,AboriginalIdentity_A16AITIDT
2,A16AITIDX,2016 Aboriginal Identities,COUNT,2016,AboriginalIdentity,AboriginalIdentity.A16AITIDX,AboriginalIdentity_A16AITIDX
3,A16AITINUK,2016 Inuk Single Identity,COUNT,2016,AboriginalIdentity,AboriginalIdentity.A16AITINUK,AboriginalIdentity_A16AITINUK
4,A16AITMETI,2016 Metis Single Identity,COUNT,2016,AboriginalIdentity,AboriginalIdentity.A16AITMETI,AboriginalIdentity_A16AITMETI
...,...,...,...,...,...,...,...
2568,P0YVISKOR,2029 VM: Korean,COUNT,2029,VisibleMinorityStatus,VisibleMinorityStatus.P0YVISKOR,VisibleMinorityStatus_P0YVISKOR
2569,P0YVISJAPA,2029 VM: Japanese,COUNT,2029,VisibleMinorityStatus,VisibleMinorityStatus.P0YVISJAPA,VisibleMinorityStatus_P0YVISJAPA
2570,P0YVISOVM,2029 VM: All Oth VM,COUNT,2029,VisibleMinorityStatus,VisibleMinorityStatus.P0YVISOVM,VisibleMinorityStatus_P0YVISOVM
2571,P0YVISMVM,2029 VM: Multiple VM,COUNT,2029,VisibleMinorityStatus,VisibleMinorityStatus.P0YVISMVM,VisibleMinorityStatus_P0YVISMVM


In [6]:
evars.drop_duplicates('name')

,name,alias,type,vintage,data_collection,enrich_str,enrich_field_name
0,A16AITFNAT,2016 First Nations Single Ident,COUNT,2016,AboriginalIdentity,AboriginalIdentity.A16AITFNAT,AboriginalIdentity_A16AITFNAT
1,A16AITIDT,2016 Aboriginal Identity,COUNT,2016,AboriginalIdentity,AboriginalIdentity.A16AITIDT,AboriginalIdentity_A16AITIDT
2,A16AITIDX,2016 Aboriginal Identities,COUNT,2016,AboriginalIdentity,AboriginalIdentity.A16AITIDX,AboriginalIdentity_A16AITIDX
3,A16AITINUK,2016 Inuk Single Identity,COUNT,2016,AboriginalIdentity,AboriginalIdentity.A16AITINUK,AboriginalIdentity_A16AITINUK
4,A16AITMETI,2016 Metis Single Identity,COUNT,2016,AboriginalIdentity,AboriginalIdentity.A16AITMETI,AboriginalIdentity_A16AITMETI
...,...,...,...,...,...,...,...
2568,P0YVISKOR,2029 VM: Korean,COUNT,2029,VisibleMinorityStatus,VisibleMinorityStatus.P0YVISKOR,VisibleMinorityStatus_P0YVISKOR
2569,P0YVISJAPA,2029 VM: Japanese,COUNT,2029,VisibleMinorityStatus,VisibleMinorityStatus.P0YVISJAPA,VisibleMinorityStatus_P0YVISJAPA
2570,P0YVISOVM,2029 VM: All Oth VM,COUNT,2029,VisibleMinorityStatus,VisibleMinorityStatus.P0YVISOVM,VisibleMinorityStatus_P0YVISOVM
2571,P0YVISMVM,2029 VM: Multiple VM,COUNT,2029,VisibleMinorityStatus,VisibleMinorityStatus.P0YVISMVM,VisibleMinorityStatus_P0YVISMVM


In [7]:
keyvars = evars[(evars.data_collection.str.startswith('Key')) & (evars.vintage == '2022')].copy().reset_index(drop=True)

keyvars

,name,alias,type,vintage,data_collection,enrich_str,enrich_field_name
0,P3YPTAPOP,2022 Total Population,COUNT,2022,KeyCanFacts,KeyCanFacts.P3YPTAPOP,KeyCanFacts_P3YPTAPOP
1,P3YHFSCF,2022 Tot Census Fam HHs,COUNT,2022,KeyCanFacts,KeyCanFacts.P3YHFSCF,KeyCanFacts_P3YHFSCF
2,P3YHSZAVG,2022 Private HHs Avg Num Persons,COUNT,2022,KeyCanFacts,KeyCanFacts.P3YHSZAVG,KeyCanFacts_P3YHSZAVG
3,P3YHNIAVG,2022 HH Inc Average Curr$,CURRENCY,2022,KeyCanFacts,KeyCanFacts.P3YHNIAVG,KeyCanFacts_P3YHNIAVG
4,P3YHNIMED,2022 HH Inc Median Curr$,CURRENCY,2022,KeyCanFacts,KeyCanFacts.P3YHNIMED,KeyCanFacts_P3YHNIMED
5,P3YTENHHD,2022 Tenure - Total HHs,COUNT,2022,KeyCanFacts,KeyCanFacts.P3YTENHHD,KeyCanFacts_P3YTENHHD
6,P3YTENOWN,2022 Tenure - Owned,COUNT,2022,KeyCanFacts,KeyCanFacts.P3YTENOWN,KeyCanFacts_P3YTENOWN
7,P3YTENRENT,2022 Tenure - Rented,COUNT,2022,KeyCanFacts,KeyCanFacts.P3YTENRENT,KeyCanFacts_P3YTENRENT
8,P3YTENBAND,2022 Tenure - Band Housing,COUNT,2022,KeyCanFacts,KeyCanFacts.P3YTENBAND,KeyCanFacts_P3YTENBAND


In [8]:
aoi_df = pd.DataFrame.spatial.from_featureclass(gdb_raw/'aoi')

aoi_df

,OBJECTID,SHAPE
0,1,"{""rings"": [[[-122.01575000099996, 49.171770000..."


In [9]:
aoi_df.spatial.plot()

MapView(layout=Layout(height='400px', width='100%'))

In [10]:
aoi_enrich_df = aoi_df.dm.enrich(keyvars, country=can)

aoi_enrich_df

,OBJECTID,KeyCanFacts_P3YPTAPOP,KeyCanFacts_P3YHFSCF,KeyCanFacts_P3YHSZAVG,KeyCanFacts_P3YHNIAVG,KeyCanFacts_P3YHNIMED,KeyCanFacts_P3YTENHHD,KeyCanFacts_P3YTENOWN,KeyCanFacts_P3YTENRENT,KeyCanFacts_P3YTENBAND,SHAPE
0,1,84903.0,23186.0,2.4,98267.9,82388.03,34079.0,24566.0,9261.0,252.0,"{""rings"": [[[-122.01575000099996, 49.171770000..."


In [11]:
aoi_enrich_fc = aoi_enrich_df.spatial.to_featureclass(gdb_int/'aoi_enrich')

aoi_enrich_fc

'D:\\projects\\canada-modeling\\data\\interim\\interim.gdb\\aoi_enrich'

In [12]:
add_enrich_aliases(aoi_enrich_fc, can)

'D:\\projects\\canada-modeling\\data\\interim\\interim.gdb\\aoi_enrich'